In [2]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


# train_xs = xs[:int(len(xs) * 0.8)]
# train_ys = ys[:int(len(xs) * 0.8)]

# val_xs = xs[-int(len(xs) * 0.2):]
# val_ys = ys[-int(len(xs) * 0.2):]

train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


## Model

In [2]:
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.tan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output


W1003 03:45:58.187588 21456 deprecation.py:506] From <ipython-input-2-08258f321552>:58: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Training dataset

In [3]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
# import driving_data
# import model

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
# train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss) #task
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        
        xs, ys = LoadTrainBatch(batch_size)
    #     train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})
        train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.5}) #task3
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
        if not os.path.exists(LOGDIR):
            os.makedirs(LOGDIR)
        checkpoint_path = os.path.join(LOGDIR, "ckpt")
        filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


W1003 03:45:58.378073 21456 deprecation_wrapper.py:119] From D:\Work\ML\API\MNB\26 - SELF DRIVING CAR\Autopilot-TensorFlow-master\Autopilot-TensorFlow-master\model.py:15: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 03:45:58.397489 21456 deprecation_wrapper.py:119] From D:\Work\ML\API\MNB\26 - SELF DRIVING CAR\Autopilot-TensorFlow-master\Autopilot-TensorFlow-master\model.py:5: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1003 03:46:00.040714 21456 deprecation.py:323] From D:\Installation\Anaconda\lib\site-packages\tensorflow\python\util\tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0, Step: 0, Loss: 12.491


W1003 03:46:08.736202 21456 saver.py:1134] *******************************************************
W1003 03:46:08.738201 21456 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1003 03:46:08.740197 21456 saver.py:1136] Consider switching to the more efficient V2 format:
W1003 03:46:08.741197 21456 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1003 03:46:08.742198 21456 saver.py:1138] now on by default.
W1003 03:46:08.743200 21456 saver.py:1139] *******************************************************


Epoch: 0, Step: 10, Loss: 11.0044
Epoch: 0, Step: 20, Loss: 9.44389
Epoch: 0, Step: 30, Loss: 8.29848
Epoch: 0, Step: 40, Loss: 7.34742
Epoch: 0, Step: 50, Loss: 6.62273
Epoch: 0, Step: 60, Loss: 6.11192
Epoch: 0, Step: 70, Loss: 6.44482
Epoch: 0, Step: 80, Loss: 6.67666
Epoch: 0, Step: 90, Loss: 4.73476
Epoch: 0, Step: 100, Loss: 4.1633


W1003 03:49:36.182972 21456 saver.py:1134] *******************************************************
W1003 03:49:36.184972 21456 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1003 03:49:36.185974 21456 saver.py:1136] Consider switching to the more efficient V2 format:
W1003 03:49:36.186975 21456 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1003 03:49:36.187975 21456 saver.py:1138] now on by default.
W1003 03:49:36.188973 21456 saver.py:1139] *******************************************************


...

Epoch: 29, Step: 3110, Loss: 0.0204789
Epoch: 29, Step: 3120, Loss: 0.0385603
Epoch: 29, Step: 3130, Loss: 0.0347054
Epoch: 29, Step: 3140, Loss: 0.0227496
Epoch: 29, Step: 3150, Loss: 0.0156126
Epoch: 29, Step: 3160, Loss: 0.0770461
Epoch: 29, Step: 3170, Loss: 0.0545484
Epoch: 29, Step: 3180, Loss: 0.234808
Epoch: 29, Step: 3190, Loss: 0.60416
Epoch: 29, Step: 3200, Loss: 0.346127


W1003 09:44:43.516077 21456 saver.py:1134] *******************************************************
W1003 09:44:43.518080 21456 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1003 09:44:43.522098 21456 saver.py:1136] Consider switching to the more efficient V2 format:
W1003 09:44:43.524085 21456 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1003 09:44:43.526081 21456 saver.py:1138] now on by default.
W1003 09:44:43.528083 21456 saver.py:1139] *******************************************************


Epoch: 29, Step: 3210, Loss: 0.108027
Epoch: 29, Step: 3220, Loss: 0.0439309
Epoch: 29, Step: 3230, Loss: 0.0120531
Epoch: 29, Step: 3240, Loss: 0.0958639
Epoch: 29, Step: 3250, Loss: 0.702784
Epoch: 29, Step: 3260, Loss: 1.51081
Epoch: 29, Step: 3270, Loss: 0.379498
Epoch: 29, Step: 3280, Loss: 0.107564
Epoch: 29, Step: 3290, Loss: 0.00458248
Epoch: 29, Step: 3300, Loss: 0.00731439


W1003 09:48:01.766767 21456 saver.py:1134] *******************************************************
W1003 09:48:01.768767 21456 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1003 09:48:01.770767 21456 saver.py:1136] Consider switching to the more efficient V2 format:
W1003 09:48:01.772773 21456 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1003 09:48:01.785765 21456 saver.py:1138] now on by default.
W1003 09:48:01.794770 21456 saver.py:1139] *******************************************************


Epoch: 29, Step: 3310, Loss: 0.0034618
Epoch: 29, Step: 3320, Loss: 0.0123202
Epoch: 29, Step: 3330, Loss: 0.040035
Epoch: 29, Step: 3340, Loss: 0.0485
Epoch: 29, Step: 3350, Loss: 0.00542531
Model saved in file: ./save\model.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser


## Run Model

In [ ]:
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0
while(cv2.waitKey(10) != ord('q')):
    



In [5]:
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

cap = cv2.VideoCapture(0)
i = 0;
while(cv2.waitKey(10) != ord('q')):
    frame = cv2.imread(val_xs[i])
    image = scipy.misc.imresize(frame, [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180 / scipy.pi
#     call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees and actual degrees"+str(val_ys[i]))
    cv2.imshow('frame', frame)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i+=1

cap.release()
cv2.destroyAllWindows()



D:\Installation\Anaconda\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Predicted steering angle: 5.247835829901312 degrees and actual degrees-0.49462631001519297
Predicted steering angle: 3.9034573795305185 degrees and actual degrees-0.5033529562751646
Predicted steering angle: 3.0433391278702175 degrees and actual degrees-0.519235452468313
Predicted steering angle: 0.04794696414308417 degrees and actual degrees-0.5420992656694387
Predicted steering angle: -1.5815633860452447 degrees and actual degrees-0.5632177496185702
Predicted steering angle: -1.9680470789094775 degrees and actual degrees-0.5843362335677015
Predicted steering angle: -6.726081292249301 degrees and actual degrees-0.6002187297608499
Predicted steering angle: -10.152889224795425 degrees and actual degrees-0.6213372137099813
Predicted steering angle: -10.062808421153067 degrees and actual degrees-0.6370451769779303
Predicted steering angle: -19.94278463105594 degrees and actual degrees-0.6564183316750674
Predicted steering angle: -24.766882355038828 degrees and actual degrees-0.67404615712

Predicted steering angle: 30.69770124344827 degrees and actual degrees-0.05637413483941684
Predicted steering angle: 33.761201595644486 degrees and actual degrees-0.08970992355250852
Predicted steering angle: 35.91027963685398 degrees and actual degrees-0.10559241974565695
Predicted steering angle: 27.019268714992055 degrees and actual degrees-0.12147491593880534
Predicted steering angle: 19.78926246908156 degrees and actual degrees-0.12671090369478832
Predicted steering angle: 25.991991188349214 degrees and actual degrees-0.12671090369478832
Predicted steering angle: 28.457272837184444 degrees and actual degrees-0.12671090369478832
Predicted steering angle: 30.30184055462726 degrees and actual degrees-0.12671090369478832
Predicted steering angle: 34.00628585853747 degrees and actual degrees-0.130201562198777
Predicted steering angle: 48.36984583399413 degrees and actual degrees-0.1319468914507713
Predicted steering angle: 62.43152852360914 degrees and actual degrees-0.1319468914507713

Predicted steering angle: -0.10123767752773724 degrees and actual degrees0.13369222070276562
Predicted steering angle: -0.24458645949636496 degrees and actual degrees0.13369222070276562
Predicted steering angle: -0.3078692827344438 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.38503820072027595 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.5600113721958709 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.5782423210293985 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.8708922347457615 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.8070517372341216 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.8813671836116223 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.7855088403442254 degrees and actual degrees0.13543754995475998
Predicted steering angle: -0.6069384003318551 degrees and actual degrees0.135

Predicted steering angle: 0.2992510514253422 degrees and actual degrees0.38013271108436497
Predicted steering angle: 0.20401060015399353 degrees and actual degrees0.37838738183237064
Predicted steering angle: 0.03251020634537975 degrees and actual degrees0.37838738183237064
Predicted steering angle: 0.09112820381343162 degrees and actual degrees0.37838738183237064
Predicted steering angle: 0.06128722739156491 degrees and actual degrees0.0
Predicted steering angle: -0.3220710602878986 degrees and actual degrees0.3960152072775134
Predicted steering angle: -0.44609993181763125 degrees and actual degrees0.404741853537485
Predicted steering angle: -0.83596862374584 degrees and actual degrees0.4082325120414737
Predicted steering angle: -1.1981991686410038 degrees and actual degrees0.4118977034706618
Predicted steering angle: -0.7509649451734743 degrees and actual degrees0.4206243497306334
Predicted steering angle: -0.07084511663964572 degrees and actual degrees0.4276056667386107
Predicted st

...

Predicted steering angle: -59.33883935799488 degrees and actual degrees0.0158824961931484
Predicted steering angle: -58.51769352540843 degrees and actual degrees0.01064650843716541
Predicted steering angle: -57.166487447188565 degrees and actual degrees0.003490658503988659
Predicted steering angle: -49.403051964836536 degrees and actual degrees-0.0017453292519943296
Predicted steering angle: -43.49232578541798 degrees and actual degrees-0.008726646259971648
Predicted steering angle: -39.41456283204238 degrees and actual degrees-0.026354471705114374
Predicted steering angle: -29.760339742134065 degrees and actual degrees-0.04223696789826277
Predicted steering angle: -23.070162619917404 degrees and actual degrees-0.05811946409141117
Predicted steering angle: -4.817235091414321 degrees and actual degrees-0.07400196028455958
Predicted steering angle: -1.8279063247362137 degrees and actual degrees-0.09512044423369097
Predicted steering angle: 6.989039733313384 degrees and actual degrees-0.1

Predicted steering angle: -8.540719547635941 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -11.210372409269004 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -13.100854365859934 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -17.70139316811914 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -23.634508707637 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -31.677522615737868 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -33.43916159137297 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -40.323343406774676 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -42.3003792280753 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -43.70440315914688 degrees and actual degrees-0.7515387759087583
Predicted steering angle: -44.87011154479295 degrees and actual degrees-0.751538775908758

Predicted steering angle: 18.35128566764585 degrees and actual degrees-0.21642082724729686
Predicted steering angle: 17.61922769995118 degrees and actual degrees-0.18133970928221083
Predicted steering angle: 18.42788965428213 degrees and actual degrees-0.16196655458507378
Predicted steering angle: 18.71200332563986 degrees and actual degrees-0.15132004614790837
Predicted steering angle: 20.891105170893432 degrees and actual degrees-0.15132004614790837
Predicted steering angle: 22.203343312351084 degrees and actual degrees0.0
Predicted steering angle: 24.760496128164867 degrees and actual degrees-0.14608405839192537
Predicted steering angle: 26.03220221954037 degrees and actual degrees-0.14433872913993104
Predicted steering angle: 26.840047966265505 degrees and actual degrees-0.14433872913993104
Predicted steering angle: 28.08096868750447 degrees and actual degrees-0.14433872913993104
Predicted steering angle: 28.505704001041714 degrees and actual degrees-0.14433872913993104
Predicted s

Predicted steering angle: 36.8256820799866 degrees and actual degrees0.052883476335428184
Predicted steering angle: 43.331638755004256 degrees and actual degrees0.052883476335428184
Predicted steering angle: 49.01458495579206 degrees and actual degrees0.052883476335428184
Predicted steering angle: 52.02454345508455 degrees and actual degrees0.052883476335428184
Predicted steering angle: 57.28107753089387 degrees and actual degrees0.052883476335428184
Predicted steering angle: 57.63461153742831 degrees and actual degrees0.052883476335428184
Predicted steering angle: 56.66485444888073 degrees and actual degrees0.052883476335428184
Predicted steering angle: 58.17173761375812 degrees and actual degrees0.052883476335428184
Predicted steering angle: 59.267887352896786 degrees and actual degrees0.052883476335428184
Predicted steering angle: 55.408789490704564 degrees and actual degrees0.052883476335428184
Predicted steering angle: 55.08622356186695 degrees and actual degrees0.0546288055874225

Predicted steering angle: 35.14866231797878 degrees and actual degrees0.12671090369478832
Predicted steering angle: 38.975169932546294 degrees and actual degrees0.12845623294678266
Predicted steering angle: 44.31503914641894 degrees and actual degrees0.1319468914507713
Predicted steering angle: 47.76949269651752 degrees and actual degrees0.1408480706359424
Predicted steering angle: 48.87115781341097 degrees and actual degrees0.1530653753999027
Predicted steering angle: 51.23338215295446 degrees and actual degrees0.1706932008450454
Predicted steering angle: 49.38899885061924 degrees and actual degrees0.1865756970381938
Predicted steering angle: 45.01770168747317 degrees and actual degrees0.19879300180215415
Predicted steering angle: 46.23281283138595 degrees and actual degrees0.19879300180215415
Predicted steering angle: 51.62095440721767 degrees and actual degrees0.19355701404617112
Predicted steering angle: 56.69916248908171 degrees and actual degrees0.18832102629018815
Predicted stee

In [14]:
# len(val_xs)
val_xs[0]

'driving_dataset/31785.jpg'

In [4]:
import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
cap = cv2.VideoCapture(0)
i = 0;
while(cv2.waitKey(10) != ord('q')):
#     ret, frame = cap.read(val_xs[i], )
    frame = cv2.imread(val_xs[i])
    cv2.imshow('image',frame)
#     cv2.waitKey()
    i+=1
    
# ret, frame = cap.read()
# image = scipy.misc.imresize(frame, [66, 200]) / 255.0

cap.release()
cv2.destroyAllWindows()

In [5]:
help(ord)

Help on built-in function ord in module builtins:

ord(c, /)
    Return the Unicode code point for a one-character string.



In [ ]:
#sigmoid will stuck 2.6085273643165015


In [5]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
# cap.release()
cv2.destroyAllWindows()